In [1]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

import numpy as np
import pandas as pd
# import requests
import datetime
import matplotlib.pyplot as plt
from utils import utils
from utils import models
import torch

In [2]:
daily=True
num_layers=3
window_size=7
test_date = "2023/9/19"
learning_rate = 5e-3


In [3]:
list_of_vars = ["temp_min", "temp_max", "temp_mean", "station"]
list_of_vars_2 = ["temp_min", "temp_max", "temp_mean", "rainfall", "snow", "station"]
list_of_vars_3 = ["temp_min", "temp_max", "temp_mean", "hist_temp_min", "hist_temp_max", "hist_temp_mean", "station"]
list_of_vars_4 = ["temp_min", "temp_max", "temp_mean", "rainfall", "snow", "hist_temp_min", "hist_temp_max",
                    "hist_temp_mean", "hist_rainfall", "hist_snow", "station"]

names_models = {0 : ["temp_min", "temp_min_hist"], 1 : ["temp_max", "temp_max_hist"], 2 : ["temp_mean", "temp_mean_hist"], 
                3 : ["rainfall", "rainfall_hist"], 4 : ["snow", "snow_hist"]}
loss_functions = {0 : torch.nn.MSELoss(), 1 : torch.nn.MSELoss(), 2 : torch.nn.MSELoss(),
    3 : models.SpecialCrossEntropyLoss(), 4 : models.SpecialCrossEntropyLoss()}
predictors_list = { 0 : [list_of_vars, list_of_vars_3], 
                    1 : [list_of_vars, list_of_vars_3],
                    2 : [list_of_vars, list_of_vars_3],
                    3 : [list_of_vars_2, list_of_vars_4],
                    4 : [list_of_vars_2, list_of_vars_4]}
test_processes = {0 : models.test_model, 1 : models.test_model, 2 : models.test_model,
    3 : models.test_model_classifier, 4 : models.test_model_classifier}

In [4]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
import os 
os.chdir("..")
data = utils.load_data(daily = daily)
train_data_pd, test_data_pd = models.train_test_split(data, test_date, daily)

In [5]:
i=0

In [6]:
train_data= models.WindowDataset(train_data_pd[predictors_list[i][0]], window_size, 4, i)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

test_data = models.WindowDataset(test_data_pd[predictors_list[i][0]], window_size, 4, i)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
x, y = next(iter(train_loader))

# Create model
inputsize = x.shape[2]
model = models.Transformer(feature_size=inputsize, 
                            num_layers=3,
                            nhead=inputsize,
                            dropout=0)
loss_function = loss_functions[i]
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
device = ("cuda" if torch.cuda.is_available() else "cpu")


In [8]:
models.train_loop(model, train_loader,  optimizer, loss_function, epochs=200)

RuntimeError: expected scalar type Double but found Float

In [9]:
# model.double()
model.train()
train_loss = 0
for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    if model.model_type == "Transformer":
        src = data.permute(1, 0, 2).to(device) # should be 7x32x3
        target = target.unsqueeze(0).permute(1, 0, 2).to(device) # should be 4x32x1
        # sampled_src = src[:1, :, :]
        output = torch.zeros_like(target)
        for i in range(len(target)):
            prediction = model(src, device)
            break 

KeyboardInterrupt: 

In [15]:
 data.permute(1,0,2).double().to(device)[:-1,:,:].shape

torch.Size([6, 32, 4])

In [16]:
prediction.shape

torch.Size([7, 32, 1])

In [25]:
target.shape[2]

4